## Get SNODAS data
Download all SNODAS SWE data for training and testing. Convert to total SWE by basin.

In [1]:
import rasterio
from rasterstats import zonal_stats

### Download and Conversion Functions

In [2]:
# Download data functions from driven data but updated to return in-memory file

from datetime import datetime
import calendar
import io
import requests

def date_url(date: datetime) -> str:
    """Get the source url for the SNODAS file on a given date"""
    url = "https://noaadata.apps.nsidc.org/NOAA/G02158/masked/"
    url += f"{date.year}/{date.month:02}_{calendar.month_abbr[date.month]}"
    url += f"/SNODAS_{date.strftime('%Y%m%d')}.tar"

    return url


def download_from_url(source_url: str) -> io.BytesIO:
    """
    Download a SNODAS file based on its source URL. 
    Returns the data TAR file as a BytesIO object.
    """
    response = requests.get(source_url)
    datatar = io.BytesIO()
    datatar.write(response.content)
    datatar.seek(0)

    return datatar

In [3]:
# Convert downloaded data to GeoTIFF stored in temporary file

import tarfile
import gzip

def saveTiffFromTar(snodas_tar, tiffname):
    '''Converts a SNODAS tar file to a GeoTIFF and saves it to disk
    
    tarfile should be in memory as a BytesIO object
    '''
    # Open TAR and extract swe related files to in memory bytes
    tar = tarfile.open(fileobj=snodas_tar, mode='r')
    for tarinfo in tar:
        if (tarinfo.name[8:12]=='1034') and (tarinfo.name[-6:]=='dat.gz'):
            #print(f'Extracting: {tarinfo.name}')
            gzfile_io = tar.extractfile(tarinfo)
            swe_zipfile = gzip.GzipFile(fileobj=gzfile_io)

        if (tarinfo.name[8:12]=='1034') and (tarinfo.name[-6:]=='txt.gz'):
            #print(f'Extracting: {tarinfo.name}')
            gzfile_io = tar.extractfile(tarinfo)
            swehdr_zipfile = gzip.GzipFile(fileobj=gzfile_io, mode='r')

    tar.close()

    # Edit one line of header file so that it points to the data file
    swehdr_lines = io.TextIOWrapper(swehdr_zipfile).readlines() #Wrapper needed to read file in text mode
    
    # Search for line that starts with 'Data file pathname:' and get the index
    pathname_index = None
    for i, line in enumerate(swehdr_lines):
        if line.startswith('Data file pathname:'):
            pathname_index = i
            break

    # Replace the line with the correct path
    if pathname_index is not None:
        swehdr_lines[pathname_index] = "Data file pathname: swe.dat\n"
    else:
        print('Error: Data file pathname not found in header file')

    with open('./data/snodas/temp/swe.dat', 'wb') as f:
        f.write(swe_zipfile.read())

    with open('./data/snodas/temp/swe.txt', 'w') as f:
        f.writelines(swehdr_lines)

    with rasterio.open('./data/snodas/temp/swe.txt') as src:
        src_profile = src.profile
        # Switch driver to tiff with compression
        src_profile.update(driver = 'GTiff',compress='lzw',)

        with rasterio.open(tiffname, 'w', **src_profile) as dst:
            dst.write(src.read())

### Initial testing

In [ ]:
swedate = datetime(2017, 4, 8)
sweurl = date_url(swedate)
swe_tar = download_from_url(sweurl)
saveTiffFromTar(swe_tar, './data/snodas/temp/swe.tif')

In [ ]:
with rasterio.open('./data/snodas/temp/swe040117test.txt') as src:
        src_profile = src.profile
        # Switch driver to tiff with compression
        src_profile.update(driver = 'GTiff',compress='lzw',)

        with rasterio.open('./data/snodas/temp/swe.tiff', 'w', **src_profile) as dst:
            dst.write(src.read())

Calculate stats and compare with QGIS

In [ ]:
stats = zonal_stats('./data/geospatial.gpkg', './data/snodas/temp/swe.tif', stats=['sum'], geojson_out=True)

For Pecos, the sum matches the sum calculated in QGIS exactly

### Determine dates for testing and training
I should only need dates between January and April since each regression will be predicting 4/1 snowpack.

In [4]:
import pandas as pd

month_days = [1,8,15,22]

# Generate a datetime index with days of each month in month_days
date_rng = pd.date_range(start='1/1/2005', end='1/1/2024', freq='D',inclusive='left')
date_rng = date_rng[date_rng.day.isin(month_days)]

# Filter values from outside the January thru July period
date_rng = date_rng[(date_rng.month >= 1) & (date_rng.month <= 4)]

### Download Data Loop
Download and process each day of data in the range. Run and save each year separately in case there is a failure and a need to restart.

In [5]:
start_year = 2017

In [8]:
from rasterio.errors import RasterioIOError

for year in range(start_year, 2024):
    date_rng_year = date_rng[date_rng.year == year]

    # Initialize list to store data
    data = []
    processed_dates = [] # Keep track of dates that have been processed in case of error

    # Download and process data for dates in date_rng_2005
    for swe_date in date_rng_year:
        print(f'Processing: {swe_date}\r', end='')
        sweurl = date_url(swe_date)
        swe_tar = download_from_url(sweurl)
        try:
            saveTiffFromTar(swe_tar, './data/snodas/temp/swe.tif')
        except RasterioIOError as rioe:
            print(f'Error: {rioe}')
            print (f'Skipping date {swe_date}')
            continue
        stats = zonal_stats('./data/geospatial.gpkg', './data/snodas/temp/swe.tif', stats=['sum'], geojson_out=True)
        processed_dates.append(swe_date)
        
        basin_swe = {}
        for basin in stats:
            basin_swe[basin['properties']['name']] = basin['properties']['sum']

        data.append(basin_swe)

    # Create dataframe from data list
    df = pd.DataFrame(data, index=pd.DatetimeIndex(processed_dates))

    # Save dataframe to CSV
    df.to_csv(f'./data/snodas/snodas_swe_{year}.csv')

Error: './data/snodas/temp/swe.txt' not recognized as a supported file format.
Skipping date 2017-04-01 00:00:00


#### Any fixes

In [7]:
date_rng_year

DatetimeIndex(['2017-01-01', '2017-01-08', '2017-01-15', '2017-01-22',
               '2017-02-01', '2017-02-08', '2017-02-15', '2017-02-22',
               '2017-03-01', '2017-03-08', '2017-03-15', '2017-03-22',
               '2017-04-01', '2017-04-08', '2017-04-15', '2017-04-22'],
              dtype='datetime64[ns]', freq=None)